# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import pandas as pd
import numpy as np

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
#pd.set_option('display.max_rows', None)
orders = pd.read_csv("Orders.csv")
orders.head()
#orders[orders.CustomerID == 12370]

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

In [4]:
#SUB PROBLEM 1
cstmr_id = orders.groupby("CustomerID").agg({"amount_spent":"sum"})
#cstmr_id

In [5]:
#SUB PROBLEM 2
per_95 = np.percentile(cstmr_id["amount_spent"], 95)
per_95
per_75 = np.percentile(cstmr_id["amount_spent"], 75)
cstmr_id_95 = cstmr_id[cstmr_id.amount_spent > per_95]
cstmr_id_75_95 = cstmr_id[(cstmr_id.amount_spent >per_75) & (cstmr_id.amount_spent <= per_95)]
cstmr_id_75_95


,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43
...,...
18259,2338.60
18260,2643.20
18272,3078.58


In [6]:
#SUB PROBLEM 3 
def label_customer(x):
    if x in cstmr_id_95.index:
        return "VIP"
    elif x in cstmr_id_75_95.index:
        return "Preferred"
    else:
        return "Normal"

In [7]:
orders["Client Label"] = orders.CustomerID.apply(label_customer)
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Client Label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here

In [9]:
vip = orders[["Country", "CustomerID", "Client Label"]]
vip = vip[vip["Client Label"] == "VIP"]

vip = vip.groupby("Country").agg({"CustomerID":pd.Series.nunique})

vip[vip["CustomerID"] == vip["CustomerID"].max()]


,CustomerID
Country,
United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
# your code here

In [11]:
v_pr = orders[["Country", "CustomerID", "Client Label"]]
v_pr = v_pr[(v_pr["Client Label"] == ("Preferred")) | (v_pr["Client Label"] == ("VIP"))]
v_pr = v_pr.groupby("Country").agg({"CustomerID": pd.Series.nunique})
v_pr[v_pr["CustomerID"] == v_pr["CustomerID"].max()]
#v_pr = v_pr.groupby("Country")["CustomerID"].nunique()


,CustomerID
Country,
United Kingdom,932


# EXTRA (PARA VER QUE CLIENTES HAN COMPRADO EN VARIOS PAISES Y EN QUE PAISES)
Hay clientes VIP y Preferred con compras en diferentes paises.
A continuación el codigo para determinar dichos clientes

In [12]:


v_pr2 = orders[["Country", "CustomerID", "Client Label"]]
v_pr2 = v_pr2[(v_pr2["Client Label"] == ("Preferred")) | (v_pr2["Client Label"] == ("VIP"))]
v_pr2 = v_pr2.groupby("CustomerID").agg({"Country": pd.Series.unique})
v_pr2.Country.head(15)
#v_pr2[v_pr2.Country.str.contains(",")]

CustomerID
12346     [United Kingdom]
12347            [Iceland]
12348            [Finland]
12349              [Italy]
12352             [Norway]
12356           [Portugal]
12357        [Switzerland]
12359             [Cyprus]
12360            [Austria]
12362            [Belgium]
12370    [Cyprus, Austria]
12371        [Switzerland]
12378        [Switzerland]
12380            [Belgium]
12381             [Norway]
Name: Country, dtype: object

In [13]:
lst = []
for e in v_pr2.Country:
    lst.append(e)
lst
count = 0
lst2 =[]
for e in lst:
    if len(e)>1:
        lst2.append(count)
    count += 1
lst2
pd.set_option('display.max_rows', None)
v_pr2.iloc[lst2]

,Country
CustomerID,
12370,"[Cyprus, Austria]"
12417,"[Belgium, Spain]"
12429,"[Denmark, Austria]"
12431,"[Australia, Belgium]"
12455,"[Cyprus, Spain]"
12457,"[Switzerland, Cyprus]"
